In [1]:
import pandas as pd

In [2]:
from helper import to_df

In [6]:
def uniform_scheme(node_id: int, nodes_df: pd.DataFrame) -> float:
    """ Assign uniform weight """
    return 1 / nodes_df.loc[nodes_df.loc[node_id, 'ParentID'], 'NKids']

In [18]:
tree = './trees/golomb_3.sqlite'
nodes_df = to_df(tree, 'nodes').set_index('NodeID')
nodes_df

,ParentID,Alternative,NKids,Status,Label
NodeID,,,,,
0,-1,-1,4,2,
1,0,0,7,2,mark[2] = 1
2,0,1,0,3,
3,0,2,0,3,
4,0,3,0,3,
5,1,0,0,0,mark[3] = 3
6,1,1,0,3,
7,1,2,0,3,
8,1,3,0,3,


In [23]:
nodes_df['NodeWeight'] = 0
nodes_df.loc[0, 'NodeWeight'] = 1 # root node has weight 1
nodes_df.loc[nodes_df[nodes_df['ParentID'] == 0].index, 'NodeWeight'] = 1 / nodes_df.loc[0, 'NKids']
next_ids = [nodes_df[nodes_df['ParentID'] == 0].index]

i, j = 0, 0

# propogate weights down
while i < len(next_ids):
    par_node = next_ids[i][j]
    if nodes_df.loc[par_node, 'NKids'] > 0:
        nodes_df.loc[nodes_df['ParentID'] == par_node, 'NodeWeight'] = \
                                nodes_df.loc[par_node, 'NodeWeight'] / nodes_df.loc[par_node, 'NKids']

        # print(f'i = {i}, j = {j}, len(next_ids) = {len(next_ids)}')
        next_ids.append(nodes_df[nodes_df['ParentID'] == par_node].index)
    j += 1
    if j >= len(next_ids[i]): 
        j = 0
        i += 1



## debug:
print(nodes_df['NodeWeight'])
print("Sum of all leaves weights: ", nodes_df[nodes_df['Status'].isin({1, 0, 3})]['NodeWeight'].sum())

NodeID
0     1.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
Name: NodeWeight, dtype: float64
Sum of all leaves weights:  0.0
